In [1]:
# Word2vec wiht ngram 111111

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.ml.classification import NaiveBayes
import numpy as np
from pyspark.sql.functions import col
from pyspark.ml.param import Param, Params
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml import Pipeline
from pyspark.sql import Row 
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import roc_curve, auc
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric

#experment Ngram wiht word2vec
from pyspark.ml.feature import NGram
data = sc.textFile("/usr/local/spark/snlpl.csv").map(lambda x:x.split("\t"))
hate = data.map(lambda x: Row(code=x[0], comment=str(x[1])))
hatedtf = sqlContext.createDataFrame(hate)
from pyspark.sql.functions import when
commentsl = hatedtf.withColumn("label", \
                           (when(col("code").like("%Positive%"), 0) \
                          # .when(col("code").like("%offensive%"), 1) \
                           .when(col("code").like("%Negative%"), 1)))#.
tokenizer = Tokenizer(inputCol="comment", outputCol="words")
wordsData = tokenizer.transform(commentsl)
ngram = NGram(n=1, inputCol="words", outputCol="ngrams")
ngramDataFrame = ngram.transform(wordsData)
word2Vec = Word2Vec(vectorSize=5, minCount=1, inputCol="ngrams", outputCol="features")
model = word2Vec.fit(ngramDataFrame)
result = model.transform(ngramDataFrame)
#file=ngramDataFrame.select("ngrams").show(truncate=True)

trainingData, testData = result.randomSplit([0.8, 0.2], seed = 100)

 #Train model with Training Data
#lrModel = lr.fit(trainingData)
rf = RandomForestClassifier(numTrees=300, maxDepth=10, labelCol="label",featuresCol = "features", seed=42)
rf_model = rf.fit(trainingData)
predictions = rf_model.transform(testData)
predictions.filter(predictions['prediction'] ==1) \
    .select("probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 5, truncate = 30)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % (accuracy))
#metrics = MulticlassClassificationEvaluator(predictions)
#print("Area under Precision/Recall (PR) curve: %.f" % (metrics.areaUnderPR * 100))
#predictions.select("label", "prediction").show()
results = predictions.select(['probability', 'label'])

## prepare score-label set
results_collect = results.collect()
results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]
scoreAndLabels = sc.parallelize(results_list)
 
metrics = metric(scoreAndLabels)
print("The ROC score is (@numTrees=300): ", metrics.areaUnderROC)
# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

fpr = dict()
tpr = dict()
roc_auc = dict()
 
y_test = [i[1] for i in results_list]
y_score = [i[0] for i in results_list]
 
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('True Positive Rate')
plt.ylabel('false Positive Rate')
plt.title('Receiver operating characteristic for Random Forest')
plt.legend(loc="lower right")
plt.show()

all_comments = predictions.select("probability").collect()
positive = [i[0][0] for i in all_comments]
#offensive = [i[0][1] for i in all_comments]
negative = [i[0][1] for i in all_comments]

# Positive
plt.hist(positive, 50, normed=1, facecolor='green', alpha=0.75)
plt.xlabel('predicted_values')
plt.ylabel('Counts')
plt.title('Probabilities for neutral cases')
plt.grid(True)
plt.show()
 
# offensive
#plt.hist(offensive, 50, normed=1, facecolor='green', alpha=0.75)
#plt.xlabel('predicted_values')
#plt.ylabel('Counts')
#plt.title('Probabilities for offensive cases')
#plt.grid(True)
#plt.show()
#Negative
plt.hist(negative, 50, normed=1, facecolor='green', alpha=0.75)
plt.xlabel('predicted_values')
plt.ylabel('Counts')
plt.title('Probabilities for hate cases')
plt.grid(True)
plt.show()
#cross validation
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [50, 100, 200]) # number of trees
             .addGrid(rf.maxDepth, [3, 4, 5]) # maximum depth
#            .addGrid(rf.maxBins, [24, 32, 40]) #Number of bins
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=10)
# Run cross validations
cvModel = cv.fit(trainingData)

# Use test set here so we can measure the accuracy of our model on new data
predictions1 = cvModel.transform(testData)
print ("result for the Cross validation")
evaluator1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator1.evaluate(predictions1)
print("Test Accuracy = %g" % (accuracy))
results1 = predictions1.select(['probability', 'label'])
results_collect = results1.collect()
results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]
scoreAndLabels1 = sc.parallelize(results_list)
 
metrics1 = metric(scoreAndLabels)
print("The ROC score is (@numTrees=200): ", metrics1.areaUnderROC)
print("Area under PR = %s" % metrics1.areaUnderPR)

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'